

We create an instance of a SimpleSkyjoEnv environment, call reset() to initialize the game and list the available agents (players):


In [1]:
import numpy as np

from rlskyjo.environment import simple_skyjo_env_v2
from importlib import reload
reload(simple_skyjo_env_v2)

<module 'rlskyjo.environment.simple_skyjo_env_v2' from '/home/michi/skybo_rl/rlskyjo/environment/simple_skyjo_env_v2.py'>

In [2]:
skyjo_env_cfg = {"num_players": 3}
env = simple_skyjo_env_v2.env(**skyjo_env_cfg)
env.reset()

env.agents, env.agent_selection

def sample_place():
    return np.random.randint(0,11)
        
def sample_draw():
    return np.random.randint(12,13)


In [3]:
print(env.agent_selection, "draw")
env.step(sample_draw())
print(env.agent_selection, "place")
env.step(
    sample_place()
)

player_1 draw
player_1 place


In [4]:
def random_admissible_policy(observation, action_mask):
    """picks randomly an admissible action from the action mask"""
    return np.random.choice(
        np.arange(len(action_mask)),
        p= action_mask/np.sum(action_mask)
    )

assert 1 not in [random_admissible_policy(None,[1,0,1]) for _ in range(1000)] 

In [6]:
env.reset()
i_episode = 1  
agent_iters = 0
while i_episode <= 1:
    for agent in env.agent_iter(max_iter=600):        
        # get observation (state) for current agent:
        obs, reward, done, info = env.last()
        
        print("training fct:", obs, reward, done, info)
        # perform q-learning with update_Q_value()
        # your code here
        
        env.render()
        if agent_iters > 600:
            break
        
        # store current state            
        if not done: 
            # choose action using epsilon_greedy_policy()
            # your code here    
            observation = obs["observations"]
            action_mask = obs["action_mask"]
            action = random_admissible_policy(observation, action_mask)
        
            print(f"sampled action {agent}: {action}")
            env.step(action)
        else: 
            # agent is done
            env.step(None)
            print('done', reward)
            break
    agent_iters = 0
    i_episode  += 1
    env.reset()

training fct: {'observations': array([15,  5, 15, 15, 15, 15, 15, 15, 15, 15, 15, 12, -2, 10, -2, 15],
      dtype=int8), 'action_mask': array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1], dtype=int8)} 0 False {}
======= render board: ===== 
======= stats ============ 
next turn: draw by Player 1 
holding card player 1: empty 
discard pile top: -2 
======= Player 0 ========== 
 [x	12	x	x]
 [x	x	x	x]
 [x	x	x	5]  
======= Player 1 ========== 
 [x	x	11	x]
 [x	x	x	x]
 [x	5	x	x]  
======= Player 2 ========== 
 [x	x	0	x]
 [x	x	x	x]
 [x	x	-2	x]  

sampled action player_1: 12
training fct: {'observations': array([15,  5, 15, 15, 15, 15, 15, 15, 15, 15, 15, 12, -2, 10, -2,  2],
      dtype=int8), 'action_mask': array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0], dtype=int8)} 0 False {}
======= render board: ===== 
======= stats ============ 
next turn: place by Player 1 
holding card player 1: 2 
discard pile top: -2 
======= Player 0 ========== 
 [x	12	x	x]
 [x	x	x	x]
 [x	x	x	5]  
======= Player 1

more envs test with rllib

In [7]:
from ray import init
init(local_mode=True)

{'node_ip_address': '172.17.87.73',
 'raylet_ip_address': '172.17.87.73',
 'redis_address': '172.17.87.73:10779',
 'object_store_address': '/tmp/ray/session_2022-01-27_19-56-08_083579_780/sockets/plasma_store',
 'raylet_socket_name': '/tmp/ray/session_2022-01-27_19-56-08_083579_780/sockets/raylet',
 'webui_url': None,
 'session_dir': '/tmp/ray/session_2022-01-27_19-56-08_083579_780',
 'metrics_export_port': 62240,
 'node_id': 'fd5d09a29e39bf0e73787ce8221f3b694e40ec9719853ef40f468ada'}

In [8]:
from ray.tune.registry import register_env
from rlskyjo.environment import simple_skyjo_env_v2
from ray.rllib.env import PettingZooEnv
import ray.rllib.agents.dqn as dqn
from ray.rllib.agents import ppo
from copy import deepcopy
from ray.rllib.agents.registry import get_agent_class
from ray.rllib.models import ModelCatalog
from rlskyjo.models.action_mask_model import TorchMaskedActions, TorchActionMaskModel
from ray.rllib.models import ModelCatalog
from ray.rllib.agents.dqn import DQNTrainer, DQNTorchPolicy
from ray.tune.logger import pretty_print
import os


In [9]:

env_name  = "pettingzoo_skyjo"
def env_creator():
        env = simple_skyjo_env_v2.env(**skyjo_env_cfg)
        return env

register_env(env_name,
                lambda config: PettingZooEnv(env_creator()))
ModelCatalog.register_custom_model(
        "pa_model2", TorchActionMaskModel
    )
env = PettingZooEnv(env_creator())

/home/michi/anaconda3/envs/skybo/lib/python3.9/site-packages/pettingzoo/utils/wrappers/base.py:44: UserWarning: The `observation_spaces` dictionary is deprecated. Use the `observation_space` function instead.
  warnings.warn("The `observation_spaces` dictionary is deprecated. Use the `observation_space` function instead.")
/home/michi/anaconda3/envs/skybo/lib/python3.9/site-packages/pettingzoo/utils/wrappers/base.py:52: UserWarning: The `action_spaces` dictionary is deprecated. Use the `action_space` function instead.
  warnings.warn("The `action_spaces` dictionary is deprecated. Use the `action_space` function instead.")


In [10]:
env.env.observe(env.env.agent_selection)

{'observations': array([15, -1, 15, 15, 15, 15, 15, 15,  9, 15, 15, 15,  8, 10,  0, 15],
       dtype=int8),
 'action_mask': array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1], dtype=int8)}

with multiagent

In [11]:
custom_config={
    "env":env_name,
    "model": {
        "custom_model": "pa_model2",
    },
    "framework": "torch",
    # Use GPUs iff `RLLIB_NUM_GPUS` env var set to > 0.
    "num_gpus": int(os.environ.get("RLLIB_NUM_GPUS", "0")),
    "num_workers": os.cpu_count(),
    "multiagent":{
            "policies": {
                name: (None, env.observation_space, env.action_space, {}) for name in env.agents
            },
            "policy_mapping_fn": lambda agent_id: agent_id
    }
}


ppo_config = ppo.DEFAULT_CONFIG.copy()
ppo_config.update(custom_config)

trainer = ppo.PPOTrainer(config=ppo_config)

2022-01-27 19:56:36,861	WARNING ppo.py:143 -- `train_batch_size` (4000) cannot be achieved with your other settings (num_workers=16 num_envs_per_worker=1 rollout_fragment_length=200)! Auto-adjusting `rollout_fragment_length` to 250.
2022-01-27 19:56:36,862	INFO ppo.py:166 -- In multi-agent mode, policies will be optimized sequentially by the multi-GPU optimizer. Consider setting simple_optimizer=True if this doesn't work for you.
2022-01-27 19:56:36,864	INFO trainer.py:743 -- Current log_level is WARN. For more information, set 'log_level': 'INFO' / 'DEBUG' or use the -v and -vv flags.
2022-01-27 19:56:36,902	WARNING worker.py:502 -- `ray.get_gpu_ids()` will always return the empty list when called from the driver. This is because Ray does not manage GPU allocations to the driver process.
:actor_name:RolloutWorker
:actor_name:RolloutWorker
:actor_name:RolloutWorker
:actor_name:RolloutWorker
:actor_name:RolloutWorker


:actor_name:RolloutWorker
:actor_name:RolloutWorker
:actor_name:RolloutWorker
:actor_name:RolloutWorker
:actor_name:RolloutWorker


:actor_name:RolloutWorker
:actor_name:RolloutWorker
:actor_name:RolloutWorker
:actor_name:RolloutWorker


:actor_name:RolloutWorker
:actor_name:RolloutWorker
:actor_name:RolloutWorker
:actor_name:RolloutWorker
:actor_name:RolloutWorker


:actor_name:RolloutWorker
:actor_name:RolloutWorker
:actor_name:RolloutWorker
:actor_name:RolloutWorker
:actor_name:RolloutWorker


:actor_name:RolloutWorker
:actor_name:RolloutWorker
:actor_name:RolloutWorker
:actor_name:RolloutWorker
:actor_name:RolloutWorker
:actor_name:RolloutWorker


:actor_name:RolloutWorker
:actor_name:RolloutWorker


In [12]:
# run manual training loop and print results after each iteration
max_steps = 1e6
max_iters = 100
for iters in range(max_iters):
    result = trainer.train()
    if iters % 2 ==0:
        print(pretty_print(result))
    # stop training if the target train steps or reward are reached
    if result["timesteps_total"] >= max_steps:
        print(f"training done, because max_steps {max_steps} {result['timesteps_total']} reached")
        break
else:
    print(f"training done, because max_iters {max_iters} reached")
# manual test loop
print("Finished training. Running manual test/inference loop.")
# prepare environment with max 10 steps


agent_timesteps_total: 3968
custom_metrics: {}
date: 2022-01-27_19-56-55
done: false
episode_len_mean: 145.73684210526315
episode_media: {}
episode_reward_max: 1.4210854715202004e-14
episode_reward_mean: -2.243819165558211e-15
episode_reward_min: -1.4210854715202004e-14
episodes_this_iter: 19
episodes_total: 19
experiment_id: 8d908192f3c54600a1a791e1afb15a5a
hostname: lenovo-michi
info:
  learner:
    player_0:
      custom_metrics: {}
      learner_stats:
        allreduce_latency: 0.0
        cur_kl_coeff: 0.19999999999999998
        cur_lr: 5.000000000000001e-05
        entropy: 1.5696350447336833
        entropy_coeff: 0.0
        kl: 0.006987270397140285
        policy_loss: -0.14716034355262916
        total_loss: 30.90483606974284
        vf_explained_var: 0.01261245866616567
        vf_loss: 31.05059880574544
      model: {}
    player_1:
      custom_metrics: {}
      learner_stats:
        allreduce_latency: 0.0
        cur_kl_coeff: 0.19999999999999998
        cur_lr: 5.0000

E0127 20:10:14.461889269    1819 backup_poller.cc:134]       Run client channel backup poller: {"created":"@1643310614.461853751","description":"pollset_work","file":"src/core/lib/iomgr/ev_epollex_linux.cc","file_line":320,"referenced_errors":[{"created":"@1643310614.461847088","description":"Bad file descriptor","errno":9,"file":"src/core/lib/iomgr/ev_epollex_linux.cc","file_line":950,"os_error":"Bad file descriptor","syscall":"epoll_wait"}]}


agent_timesteps_total: 363968
custom_metrics: {}
date: 2022-01-27_20-10-36
done: false
episode_len_mean: 1450.08
episode_media: {}
episode_reward_max: 1.4210854715202004e-14
episode_reward_mean: -5.329070518200751e-16
episode_reward_min: -1.4210854715202004e-14
episodes_this_iter: 2
episodes_total: 615
experiment_id: 8d908192f3c54600a1a791e1afb15a5a
hostname: lenovo-michi
info:
  learner:
    player_0:
      custom_metrics: {}
      learner_stats:
        allreduce_latency: 0.0
        cur_kl_coeff: 0.15
        cur_lr: 5.000000000000001e-05
        entropy: 0.8091247719526291
        entropy_coeff: 0.0
        kl: 0.01162297041501006
        policy_loss: -0.07666398223799964
        total_loss: 5.942584645152092
        vf_explained_var: -0.1222428168853124
        vf_loss: 6.0175051950414975
      model: {}
    player_1:
      custom_metrics: {}
      learner_stats:
        allreduce_latency: 0.0
        cur_kl_coeff: 0.19999999999999998
        cur_lr: 5.000000000000001e-05
        

In [21]:
env = PettingZooEnv(env_creator())
obs = env.reset()
done = {'__all__': False}
# run one iteration until done
while not done["__all__"]:
    agent = env.env.agent_selection
    obs = env.env.observe(agent)
    env.render()
    
    # get information on action
    policy = trainer.get_policy(policy_id=agent)
    action_off_policy, _, action_info = policy.compute_single_action(obs, policy_id=agent)
    logits = action_info['action_dist_inputs']
    action = logits.argmax()
    print("agent ", agent, " action ", logits.argmax())
    next_obs, reward, done, _ = env.step({agent: action})
    # observations contain original observations and the action mask
    # print(f"Obs: {obs}, Action: {action}, done: {done}")
env.render()
    

======= render board: ===== 
======= stats ============ 
next turn: draw by Player 0 
holding card player 0: empty 
discard pile top: 9 
======= Player 0 ========== 
 [x	3	x	x]
 [x	x	x	x]
 [x	7	x	x]  
======= Player 1 ========== 
 [x	x	x	x]
 [x	2	x	x]
 [0	x	x	x]  
======= Player 2 ========== 
 [x	x	-1	x]
 [x	x	x	x]
 [8	x	x	x]  

agent  player_0  action  13


/home/michi/anaconda3/envs/skybo/lib/python3.9/site-packages/pettingzoo/utils/wrappers/base.py:44: UserWarning: The `observation_spaces` dictionary is deprecated. Use the `observation_space` function instead.
  warnings.warn("The `observation_spaces` dictionary is deprecated. Use the `observation_space` function instead.")
/home/michi/anaconda3/envs/skybo/lib/python3.9/site-packages/pettingzoo/utils/wrappers/base.py:52: UserWarning: The `action_spaces` dictionary is deprecated. Use the `action_space` function instead.
  warnings.warn("The `action_spaces` dictionary is deprecated. Use the `action_space` function instead.")


TypeError: '<=' not supported between instances of 'int' and 'tuple'

In [ ]:
obs = {'observation': np.array([15, 15,  7,  4, 15, -2, 15,  6, 12, 15, 15,  4,  9,  6,  3,  0,  1,
        0,  1,  0,  2,  1,  1,  2,  0,  1,  1,  0,  1, -2,  4], dtype=np.int8), 'action_mask': np.array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0], dtype=np.int8)}
action = trainer.compute_single_action(obs, policy_id=agent)

Try creating with rllib

In [ ]:
alg_name = "DQN"


config = deepcopy(get_agent_class(alg_name)._default_config)

register_env(env_name,
                lambda config: PettingZooEnv(env_creator()))

ModelCatalog.register_custom_model(
        "pa_model", TorchMaskedActions
    )
config["model"] = {
            "custom_model": "pa_model",
        }
config["framework"] = "torch"

In [ ]:
sample_env = PettingZooEnv(env_creator())
obs_space = sample_env.observation_space
act_space = sample_env.action_space

config["multiagent"] = {
    "policies": {
        name: (None, obs_space, act_space, {}) for name in sample_env.agents
    },
    "policy_mapping_fn": lambda agent_id: agent_id
}

In [ ]:
config["num_workers"] = 0
config['env'] = env_name